In [102]:
import torch
from transformers import BertTokenizerFast

fpath='data/tokenizer_model'
tokenizer = BertTokenizerFast.from_pretrained(fpath,
                                              strip_accents=False,
                                              lowercase=False)

In [115]:
class BERTLangaugeModelDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_seq_len=128, masking_ratio=0.15, NSP_ratio=0.5):
        super(BERTLangaugeModelDataset, self).__init__()

        self.data          = data        
        self.tokenizer     = tokenizer
        self.vocab         = tokenizer.vocab
        self.max_seq_len   = max_seq_len
        self.masking_ratio = masking_ratio
        self.NSP_ratio     = NSP_ratio
        
        self.cls_token_id  = self.tokenizer.cls_token_id
        self.sep_token_id  = self.tokenizer.sep_token_id
        self.pad_token_id  = self.tokenizer.pad_token_id
        self.mask_token_id = self.tokenizer.mask_token_id
        
    def __getitem__(self, sent_1_idx):       
        sent_1 = self.tokenizer.encode(self.data[sent_1_idx])[1:-1]
        sent_2_idx = sent_1_idx + 1
        
        # NSP
        if torch.rand(1) >= self.NSP_ratio:
            sent_2 = self.tokenizer.encode(self.data[sent_1_idx + 1])[1:-1]
            is_next = torch.tensor(1)
        else:
            while sent_2_idx == sent_1_idx + 1:
                sent_2_idx = torch.randint(0, len(self.data), (1,))
            is_next = torch.tensor(0)

        sent_2 = self.tokenizer.encode(self.data[sent_2_idx])[1:-1]
        
        # if length of (sent 1 + sent 2) longer than threshold
        # CLS, SEP 1 and 2
        if len(sent_1) + len(sent_2) >= self.max_seq_len - 3:
            sent_2 = sent_2[:self.max_seq_len - 3 - len(sent_1)]
        
        pad_length = self.max_seq_len - 3 - len(sent_1) - len(sent_2)
        target = torch.tensor([self.cls_token_id] + sent_1 + [self.sep_token_id] + sent_2 + [self.sep_token_id] + [self.pad_token_id] * pad_length).long().contiguous()        

        sent_embedding = torch.zeros(target.size(0))
        sent_embedding[(len(sent_1) + 2):] = 1
        
        # MLM
        train = torch.cat([
            torch.tensor([self.cls_token_id]), 
            self.masking(sent_1),
            torch.tensor([self.sep_token_id]),
            self.masking(sent_2),
            torch.tensor([self.sep_token_id]),
            torch.tensor([self.pad_token_id] * pad_length)
        ]).long().contiguous()
        
        return train, target, sent_embedding, is_next
        
    
    def __len__(self):
        return len(self.data)
    
    
    def __iter__(self):
        for x in self.data:
            yield x
            
    
    def get_vocab(self):
        return self.vocab
    
    
    def decode(self, x):
        return self.tokenizer.batch_decode(x)
    
    
    def masking(self, x):
        x = torch.tensor(x).long().contiguous()
        masking_idx   = torch.randperm(x.size()[0])[:round(x.size()[0] * self.masking_ratio) + 1]       
        masking_label = torch.zeros(x.size()[0])
        masking_label[masking_idx] = 1
        x = x.masked_fill(masking_label.bool(), self.mask_token_id)
        
        return x  

In [116]:
with open("data/petitions.txt", 'r') as f:
    data = f.readlines()

    
proced_data = [line.replace("\n", "") for line in data]
# proced_data = []
# for line in data:
#     proced_data.append(line.replace("\n", "").split(" "))

In [117]:
proced_data[:10]

['국민과 소통하시고 자유롭고 행복한 나라를 만들기 위해 힘쓰고 계신 대통령께 존경과 찬사를 올립니다.',
 '기해년 새해 복 많이 받으십시오.',
 '저는 경북 울진군 북면 부구검성로 12번지에 살고 있는 북면발전협의회장 이희국이라고 합니다.',
 '저는 8기의 원전이 가동․건설되고 있는 이곳 북면에 태어나 68년째 거주하고 있는 원전지역 주민입니다.',
 '간절한 마음을 담아 대통령께 다음과 같이 호소 드립니다.',
 '‘울진군민과 약속한 신한울 3,4호기 원전건설을 재개해 주십시오.’ 여태껏 단 한 번도 원전 건설을 원한 적 없는 제가 신한울 3,4호기 원전 건설을 청하는 까닭을 말씀드리겠습니다.',
 '경상북도 동해안 최북단 울진군은 예부터 산과 바다, 계곡의 울창함이 보배처럼 아름답다하여 “울진(蔚珍)”이라는 지명을 간직하게 된 곳입니다.',
 '이러한 곳에 1981년 원전사업의 시작으로 울진군에 북면(6기), 산포지구(6기), 직산지구(6기)가 원전 예정지역으로 지정되면서, 먼저 북면 부구리 지역에 원전 6기가 건설되었습니다.',
 '해안선이 잘려나가고 마을 한복판에 고압 송전탑이 들어섰습니다.',
 '어장이 파괴되고 지역 특산품에 방사능 꼬리표가 붙었습니다.']

In [148]:
tokenizer.convert_ids_to_tokens(tokenizer.encode(proced_data[0]))

['[CLS]',
 '국민',
 '##과',
 '소통',
 '##하',
 '##시고',
 '자유',
 '##롭',
 '##고',
 '행복',
 '##한',
 '나라',
 '##를',
 '만들',
 '##기',
 '위해',
 '힘쓰',
 '##고',
 '계신',
 '대통령',
 '##께',
 '존경',
 '##과',
 '찬사',
 '##를',
 '올립니다',
 '.',
 '[SEP]']

In [149]:
tokenizer.decode(tokenizer.encode(proced_data[0]))

'[CLS] 국민과 소통하시고 자유롭고 행복한 나라를 만들기 위해 힘쓰고 계신 대통령께 존경과 찬사를 올립니다. [SEP]'

In [146]:
proced_data[0]

'국민과 소통하시고 자유롭고 행복한 나라를 만들기 위해 힘쓰고 계신 대통령께 존경과 찬사를 올립니다.'

In [141]:
tokenizer_torch = tokenizer("나는 오늘 아침밥을 먹었다.", return_tensors="pt")
print("Tokens (str)      : {}".format([tokenizer.convert_ids_to_tokens(s) for s in tokenizer_torch['input_ids'].tolist()[0]]))




Tokens (str)      : ['[CLS]', '나', '##는', '오늘', '아침밥', '##을', '먹', '##었', '##다', '.', '[SEP]']


In [125]:
tokenizer.decode(tokenizer.encode(proced_data[0]))

'[CLS] 국민과 소통하시고 자유롭고 행복한 나라를 만들기 위해 힘쓰고 계신 대통령께 존경과 찬사를 올립니다. [SEP]'

In [123]:
dataset = BERTLangaugeModelDataset(data=proced_data, tokenizer=tokenizer)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=5, shuffle=True)


for batch, (mlm_train, mlm_target, sent_emb, is_next) in enumerate(data_loader):
#     print(mlm_train)
    print(tokenizer.batch_decode(mlm_train))
#     print(mlm_target)
    print(tokenizer.batch_decode(mlm_target))
    print(sent_emb)
    print(is_next)
    break
    

['[CLS] 3년차 [MASK]벗 [MASK] [MASK]는 더 초심으로 돌아가 개혁 혁신 [MASK] 이루 [MASK] 지시길 기분좋게 만들어 보시길 잘 ㅎ ~ 👍😃 [MASK] [SEP] 저는 너무 황당 [MASK] 겁도 나고 [MASK]였지만 화가나서 그 차 옆을 지날때 크락션 [MASK] [MASK]고 갔 [MASK] 저를 따라온 그 [MASK] [MASK]는 제 옆차선에 [MASK] 욕을 하기 시작했습니다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] [MASK] 기간 [MASK]료 [MASK] [MASK] 12월 31일 퇴근직전까지도 누가 살아남고 누가 짤리는지 [MASK] 모릅니다. [SEP] [MASK]. * * * 소장 [MASK] 1 ) 계약서 [MASK] 잘주지 않는다 [MASK] [MASK]에 싸인만 이곳저곳 하게끔 하고는 회사직 [MASK]을 찍고 나서 나중에 다시 [MASK]에 준다고 이야기 한다 [MASK] [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P